In [1]:
import numpy as np
import pandas as pd
import pickle5 as pickle

In [2]:
with open("../data/building_df_reduced.pkl","rb") as f:
    building_df = pickle.load(f)

building_df

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,7432,2008,0
1,0,1,0,2720,2004,0
2,0,2,0,5376,1991,0
3,0,3,0,23685,2002,0
4,0,4,0,116607,1975,0
...,...,...,...,...,...,...
1444,15,1444,1,19619,1914,0
1445,15,1445,0,4298,-999,0
1446,15,1446,1,11265,1997,0
1447,15,1447,10,29775,2001,0


In [3]:
building_df.drop(columns="floor_count", inplace=True)
building_df

,site_id,building_id,primary_use,square_feet,year_built
0,0,0,0,7432,2008
1,0,1,0,2720,2004
2,0,2,0,5376,1991
3,0,3,0,23685,2002
4,0,4,0,116607,1975
...,...,...,...,...,...
1444,15,1444,1,19619,1914
1445,15,1445,0,4298,-999
1446,15,1446,1,11265,1997
1447,15,1447,10,29775,2001


In [4]:
with open("../data/train_weather_reduced.pkl", "rb") as f:
    train_weather = pickle.load(f)
    
train_weather

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
...,...,...,...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6


### Timestamp alignment

In [5]:
import datetime

max_temp_hrs = []
for site in range(0,15+1):
    day = pd.Timestamp(datetime.datetime(2016, 1, 1))
    max_temp_hrs_per_day = []
    
    for _ in range(366):
        day_after = day + pd.Timedelta("1 day")
        df = train_weather[(train_weather["site_id"]==site) & (train_weather["timestamp"] >= day) & (train_weather["timestamp"] < day_after)][["timestamp", "air_temperature"]]
        loc = df["air_temperature"].idxmax()
        hour = df.loc[loc]["timestamp"].hour 
        day = day + pd.Timedelta("1 day")
        max_temp_hrs_per_day.append(hour)
    
    max_temp_hrs.append(max_temp_hrs_per_day)

In [6]:
from scipy.stats import mode

# We align so that all the mode of each sites peak temp time
# has to occur during the afternoon 13:00 to 15:00 time


max_temp = mode(np.array(max_temp_hrs),axis=1).mode.flatten()
sites = ["site_{}".format(i) for i in range(0,16)]
alignment = pd.DataFrame(zip(sites, max_temp),columns=["site_id","peak_temp_hr"])

def correction(hr):
    # peak afternoon hrs are not corrected
    if hr==13 or hr==14 or hr==15:
        return 0
    
    # These sites are in mountain locs and we set peak at 16:00 by -8
    elif hr==0:
        return -8
    
    # For rest of the sites, we align to 15:00
    else:
        return 15-hr

alignment["correction"] = alignment["peak_temp_hr"].apply(correction)
alignment

,site_id,peak_temp_hr,correction
0,site_0,19,-4
1,site_1,14,0
2,site_2,0,-8
3,site_3,20,-5
4,site_4,0,-8
5,site_5,13,0
6,site_6,20,-5
7,site_7,21,-6
8,site_8,19,-4
9,site_9,21,-6


In [7]:
# Algning timezone for test data
train_weather_copy = train_weather.copy()
for site in range(0, 16):
    df = train_weather[train_weather["site_id"]==site]["timestamp"].apply(lambda x:x+pd.Timedelta("{} hr".format(alignment.iloc[site]["correction"])))
    train_weather_copy.loc[train_weather_copy["site_id"]==site, "timestamp"] = df

train_weather = train_weather_copy
del train_weather_copy

In [8]:
with open("../data/train_reduced.pkl", "rb") as f:
    train_reduced = pickle.load(f)

train_reduced

,building_id,meter,timestamp,meter_reading,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,DT_week_month
0,0,0,2016-01-01 00:00:00,0.000,1,53,1,0,4,1,1
1,1,0,2016-01-01 00:00:00,0.000,1,53,1,0,4,1,1
2,2,0,2016-01-01 00:00:00,0.000,1,53,1,0,4,1,1
3,3,0,2016-01-01 00:00:00,0.000,1,53,1,0,4,1,1
4,4,0,2016-01-01 00:00:00,0.000,1,53,1,0,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750,12,52,366,23,5,31,5
20216096,1445,0,2016-12-31 23:00:00,4.825,12,52,366,23,5,31,5
20216097,1446,0,2016-12-31 23:00:00,0.000,12,52,366,23,5,31,5
20216098,1447,0,2016-12-31 23:00:00,159.575,12,52,366,23,5,31,5


In [9]:
building_df[building_df["site_id"]==0]

,site_id,building_id,primary_use,square_feet,year_built
0,0,0,0,7432,2008
1,0,1,0,2720,2004
2,0,2,0,5376,1991
3,0,3,0,23685,2002
4,0,4,0,116607,1975
...,...,...,...,...,...
100,0,100,10,24456,1968
101,0,101,12,18860,1986
102,0,102,12,15876,1983
103,0,103,0,21657,2016


### Dropping few site0 data

In [10]:
# Almost all of readings from this site are zeros in the beggining of the year
# We pick an arbitary date so that most no. of total readings are zeros and drop them from
# the train data.

day = pd.Timestamp(datetime.datetime(2016, 5, 25))
df = train_reduced[(train_reduced["building_id"] < 105) & (train_reduced["timestamp"]<day)]
print("Total no. readings {}".format(len(df)))
print("Total no. of zero reading meters {}".format((df["meter_reading"]==0).sum()))

Total no. readings 404822
Total no. of zero reading meters 350360


In [11]:
train_reduced.drop(df.index, inplace=True)

### Hour cyclical + weekend feature

In [12]:
train_reduced["DT_h_sin"] = train_reduced["DT_hour"].apply(lambda x:np.sin(2*np.pi*x/24))
train_reduced["DT_h_cos"] = train_reduced["DT_hour"].apply(lambda x:np.cos(2*np.pi*x/24))

# Week days are coded as Mon:0,Tue:1,...Fri:4,Sat:5,Sun:6
train_reduced["weekend"] = train_reduced["DT_day_week"].apply(lambda x: int(x>=5))

train_reduced

,building_id,meter,timestamp,meter_reading,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,DT_week_month,DT_h_sin,DT_h_cos,weekend
103,105,0,2016-01-01 00:00:00,23.3036,1,53,1,0,4,1,1,0.000000,1.000000,0
104,106,0,2016-01-01 00:00:00,0.3746,1,53,1,0,4,1,1,0.000000,1.000000,0
105,106,3,2016-01-01 00:00:00,0.0000,1,53,1,0,4,1,1,0.000000,1.000000,0
106,107,0,2016-01-01 00:00:00,175.1840,1,53,1,0,4,1,1,0.000000,1.000000,0
107,108,0,2016-01-01 00:00:00,91.2653,1,53,1,0,4,1,1,0.000000,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.7500,12,52,366,23,5,31,5,-0.258819,0.965926,1
20216096,1445,0,2016-12-31 23:00:00,4.8250,12,52,366,23,5,31,5,-0.258819,0.965926,1
20216097,1446,0,2016-12-31 23:00:00,0.0000,12,52,366,23,5,31,5,-0.258819,0.965926,1
20216098,1447,0,2016-12-31 23:00:00,159.5750,12,52,366,23,5,31,5,-0.258819,0.965926,1


In [13]:
train_featurized = train_reduced.merge(building_df, on="building_id", how="left").merge(
    train_weather, on=["site_id","timestamp"], how="left")
train_featurized

,building_id,meter,timestamp,meter_reading,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,...,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,105,0,2016-01-01 00:00:00,23.3036,1,53,1,0,4,1,...,0,50623,-999,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
1,106,0,2016-01-01 00:00:00,0.3746,1,53,1,0,4,1,...,0,5374,-999,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
2,106,3,2016-01-01 00:00:00,0.0000,1,53,1,0,4,1,...,0,5374,-999,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
3,107,0,2016-01-01 00:00:00,175.1840,1,53,1,0,4,1,...,0,97532,2005,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
4,108,0,2016-01-01 00:00:00,91.2653,1,53,1,0,4,1,...,0,81580,1913,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19811273,1444,0,2016-12-31 23:00:00,8.7500,12,52,366,23,5,31,...,1,19619,1914,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19811274,1445,0,2016-12-31 23:00:00,4.8250,12,52,366,23,5,31,...,0,4298,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19811275,1446,0,2016-12-31 23:00:00,0.0000,12,52,366,23,5,31,...,1,11265,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19811276,1447,0,2016-12-31 23:00:00,159.5750,12,52,366,23,5,31,...,10,29775,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
with open("../data/train_featurized.pkl", "wb") as f:
    pickle.dump(train_featurized, f)


In [15]:
with open("../data/test_reduced.pkl", "rb") as f:
    test_reduced = pickle.load(f)

test_reduced

,row_id,building_id,meter,timestamp,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,DT_week_month
0,0,0,0,2017-01-01 00:00:00,1,52,1,0,6,1,1
1,1,1,0,2017-01-01 00:00:00,1,52,1,0,6,1,1
2,2,2,0,2017-01-01 00:00:00,1,52,1,0,6,1,1
3,3,3,0,2017-01-01 00:00:00,1,52,1,0,6,1,1
4,4,4,0,2017-01-01 00:00:00,1,52,1,0,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...
41697595,41697595,1444,0,2018-05-09 07:00:00,5,19,129,7,2,9,2
41697596,41697596,1445,0,2018-05-09 07:00:00,5,19,129,7,2,9,2
41697597,41697597,1446,0,2018-05-09 07:00:00,5,19,129,7,2,9,2
41697598,41697598,1447,0,2018-05-09 07:00:00,5,19,129,7,2,9,2


In [16]:
test_reduced["DT_h_sin"] = test_reduced["DT_hour"].apply(lambda x:np.sin(2*np.pi*x/24))
test_reduced["DT_h_cos"] = test_reduced["DT_hour"].apply(lambda x:np.cos(2*np.pi*x/24))
test_reduced["weekend"] = test_reduced["DT_day_week"].apply(lambda x: int(x>=5))

test_reduced

,row_id,building_id,meter,timestamp,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,DT_week_month,DT_h_sin,DT_h_cos,weekend
0,0,0,0,2017-01-01 00:00:00,1,52,1,0,6,1,1,0.000000,1.000000,1
1,1,1,0,2017-01-01 00:00:00,1,52,1,0,6,1,1,0.000000,1.000000,1
2,2,2,0,2017-01-01 00:00:00,1,52,1,0,6,1,1,0.000000,1.000000,1
3,3,3,0,2017-01-01 00:00:00,1,52,1,0,6,1,1,0.000000,1.000000,1
4,4,4,0,2017-01-01 00:00:00,1,52,1,0,6,1,1,0.000000,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41697595,41697595,1444,0,2018-05-09 07:00:00,5,19,129,7,2,9,2,0.965926,-0.258819,0
41697596,41697596,1445,0,2018-05-09 07:00:00,5,19,129,7,2,9,2,0.965926,-0.258819,0
41697597,41697597,1446,0,2018-05-09 07:00:00,5,19,129,7,2,9,2,0.965926,-0.258819,0
41697598,41697598,1447,0,2018-05-09 07:00:00,5,19,129,7,2,9,2,0.965926,-0.258819,0


In [17]:
with open("../data/test_weather_reduced.pkl", "rb") as f:
    test_weather = pickle.load(f)

In [18]:
# timestamp alignment of test data
test_weather_copy = test_weather.copy()
for site in range(0, 16):
    df = test_weather[test_weather["site_id"]==site]["timestamp"].apply(lambda x:x+pd.Timedelta("{} hr".format(alignment.iloc[site]["correction"])))
    test_weather_copy.loc[test_weather_copy["site_id"]==site, "timestamp"] = df

test_weather = test_weather_copy
del test_weather_copy

In [19]:
test_featurized = test_reduced.merge(building_df, on="building_id", how="left").merge(
    test_weather, on=["site_id","timestamp"], how="left")
test_featurized

,row_id,building_id,meter,timestamp,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,...,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,1,52,1,0,6,1,...,0,7432,2008,16.7,2.0,13.3,0.0,1022.3,130.0,2.6
1,1,1,0,2017-01-01 00:00:00,1,52,1,0,6,1,...,0,2720,2004,16.7,2.0,13.3,0.0,1022.3,130.0,2.6
2,2,2,0,2017-01-01 00:00:00,1,52,1,0,6,1,...,0,5376,1991,16.7,2.0,13.3,0.0,1022.3,130.0,2.6
3,3,3,0,2017-01-01 00:00:00,1,52,1,0,6,1,...,0,23685,2002,16.7,2.0,13.3,0.0,1022.3,130.0,2.6
4,4,4,0,2017-01-01 00:00:00,1,52,1,0,6,1,...,0,116607,1975,16.7,2.0,13.3,0.0,1022.3,130.0,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41697595,41697595,1444,0,2018-05-09 07:00:00,5,19,129,7,2,9,...,1,19619,1914,16.1,4.0,10.0,NaN,1020.5,110.0,2.6
41697596,41697596,1445,0,2018-05-09 07:00:00,5,19,129,7,2,9,...,0,4298,-999,16.1,4.0,10.0,NaN,1020.5,110.0,2.6
41697597,41697597,1446,0,2018-05-09 07:00:00,5,19,129,7,2,9,...,1,11265,1997,16.1,4.0,10.0,NaN,1020.5,110.0,2.6
41697598,41697598,1447,0,2018-05-09 07:00:00,5,19,129,7,2,9,...,10,29775,2001,16.1,4.0,10.0,NaN,1020.5,110.0,2.6


In [20]:
with open("../data/test_featurized.pkl", "wb") as f:
    pickle.dump(test_featurized, f)
